In [1]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_df = pd.read_csv('train.csv')
train_df['Answer']

0      2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년...
1      2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7...
2      기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정...
3      일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별...
4      2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216...
                             ...                        
491    자치단체 보조금은 2019년 대비 2022년에 35% 증가하였고, 민간보조금은 10...
492    한국의 재정금융안정계획은 통화량 조절과 물가안정이라는 단기적 목표에 초점을 맞추어 ...
493    원조물자 판매대금의 효과적 활용, 참전유엔군 경비지출을 위해 우리 정부에서 대여해 ...
494    IMF는 최근 세계 경제전망을 통해 작년 3.5%에서 올해 3%, 내년 2.9%로 ...
495    한국의 재정금융안정계획은 통화량 조절과 물가안정이라는 단기적 목표에 초점을 맞추어 ...
Name: Answer, Length: 496, dtype: object

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

def ngram_analysis(df, n=2):
    # 답변 텍스트를 리스트로 변환
    texts = df['Answer'].tolist()

    # CountVectorizer를 사용해 N-gram 추출
    vectorizer = CountVectorizer(ngram_range=(n, n))
    X = vectorizer.fit_transform(texts)

    # N-gram과 그 빈도수 출력
    ngram_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    ngram_sums = ngram_counts.sum().sort_values(ascending=False)
    
    return ngram_sums

# N-gram 분석 결과 출력 (2-gram 분석)
ngram_results = ngram_analysis(train_df, n=1)
ngram_results

위해      61
있다      57
따라      45
있습니다    44
예산은     43
        ..
보인       1
보인다      1
보장       1
보장과      1
희생과      1
Length: 3893, dtype: int64

In [2]:
def setup_llm_pipeline(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 토크나이저 로드 및 설정
        # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,low_cpu_mem_usage=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        temperature = 0.5,
        max_new_tokens=1024,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.04s/it]


In [4]:
def template_extractor(df):
    full_template = "<|begin_of_text|>"
    full_template += """<|start_header_id|>system<|end_header_id|>아래 문장들에서 나타나는 답변 포맷들을 찾아주세요.<|eot_id|>"""
    for i, answer in enumerate(train_df['Answer']):
        full_template += f"ex{i}: {answer}<|eot_id|>\n"
    full_template += """<|start_header_id|>user<|end_header_id|>{input}<|eot_id|>"""
    full_template += f"<|start_header_id|>assistant<|end_header_id>"
    template = PromptTemplate(template=full_template,input_variables=["input"])
    chain =({
        "input": RunnablePassthrough(),
    }
        | template
        | llm
        | StrOutputParser()
    )
    result = chain.invoke(input="문장들에서 나타나는 답변 문체들을 찾아주세요.")
    return result

results =[]
for i in range(0, 496,4):
    results.append(template_extractor(train_df.iloc[i:i+4]))

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.55 GiB. GPU 